In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


url = "https://alternative-fuels-observatory.ec.europa.eu/general-information/news/eafo-analysis-trends-ev-charging-infrastructure-across-europe"

response = requests.get(url)


soup = BeautifulSoup(response.content, 'html.parser')


rows = soup.find_all('tr', class_='ecl-table__row')


headers = []
header_row = soup.find('tr')
if header_row:
    headers = [th.text.strip() for th in header_row.find_all('th')]


data = []
for row in rows:
    cols = [col.text.strip() for col in row.find_all('td')]
    if cols:
        data.append(cols)


df = pd.DataFrame(data, columns=headers if headers else None)



df.to_csv('scraped_rows.csv', index=False)

                 0                                    1  \
0          Country  Recharging Power / Recharging Point   
1          Austria                                30,74   
2          Belgium                                27,51   
3         Bulgaria                                50,44   
4          Croatia                                42,57   
5           Cyprus                                20,03   
6   Czech Republic                                39,30   
7          Denmark                                34,60   
8          Estonia                                51,81   
9          Finland                                42,18   
10          France                                30,39   
11         Germany                                40,68   
12          Greece                                26,45   
13         Hungary                                34,13   
14         Iceland                                39,90   
15         Ireland                                36,23 